In [1]:
pip install sentence-transformers



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Step 1: Cleans problem tags, 
# Step 2: Constructs problem matrices by consolidating district-problem groups
# Step 3: Generates semantics oriented embeddings for solutions, used in problem solution matching
# Step 4: Identifies district problems which have no current solutions

In [55]:
# libraries
import pandas as pd
from ast import literal_eval
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from itertools import chain

#-------------------------------------------------------------
# vars and loading the data

# model recommended from https://medium.com/@rahultiwari065/unlocking-the-power-of-sentence-embeddings-with-all-minilm-l6-v2-7d6589a5f0aa and https://mlflow.org/docs/latest/llms/sentence-transformers/
SENTENCE_MODEL = SentenceTransformer('all-MiniLM-L6-v2') 

CASE_EXAMPLE_FILENAME = 'prob_solution_50_district_example.csv' #'prob_solution_5_district_example.csv'
tagged_district_df = pd.read_csv(CASE_EXAMPLE_FILENAME)

DEEPSEEK_RESULTS = "deepseek_results_24_25_only.csv" # the deepseek CSV relevancy results with dates
metadata_df = pd.read_csv(DEEPSEEK_RESULTS)[["leaid","place_name","state_name","centroid_lat","centroid_lon"]].drop_duplicates()


#-------------------------------------------------------------

# CONSOLIDATION FUNCTIONS 

def clean_problem_tag(problem):
    try: #needed to fix double quotes, leading + trailing quotes
        return literal_eval(problem) 
        
    except (ValueError, SyntaxError):
        return problem


def consolidate_problem_group(problem,group_df):
    '''
    consolidates problem group dataframe into a single row df
    '''

    updated_df = group_df.iloc[[0]].copy()
    
    # consolidate as needed
    if len(group_df) ==1:
        return updated_df
        
    else: 
        # only need to combine solution lists
        all_lists = updated_df['solutions_list'].tolist()

        #make into flattened list of solutions
        flattened = list(chain.from_iterable(all_lists))

        updated_df.at[updated_df.index[0], 'solutions_list'] = flattened
        
        return updated_df


def consolidate_district_problem_df(tagged_district_df):

    updated_district_prob_lst = []
    
    problem_groups = tagged_district_df.groupby(['leaid','problem_tag'])
    
    for id_prob_tuple, group_df in problem_groups:
        # combine solutions lists for the same problems within the same district
        updated_district_prob_df = consolidate_problem_group(id_prob_tuple[1],group_df)
        updated_district_prob_lst.append(updated_district_prob_df)
            
    updated_problem_df = pd.concat(updated_district_prob_lst).reset_index(drop=True)
    return updated_problem_df



# PROBLEM SOLUTION FUNCTIONS 

def get_solution_vectors(solution_lst):
    if len(solution_lst)!=0:
        single_vectors = SENTENCE_MODEL.encode(solution_lst)
        return np.mean(single_vectors, axis=0)
    else:
        return 'NO_SOLUTIONS'


def find_closest_and_furthest(district_to_match, df):
    '''
    get the closest + furthest solution match from the given district
    '''

    this_districts_vector = df[df['leaid'] == district_to_match]['solution_vector'].iloc[0]
    this_districts_solutions = df[df['leaid'] == district_to_match]['solutions_list'].iloc[0]

    this_districts_place = df[df['leaid'] == district_to_match]['place_name'].iloc[0]
    this_districts_state = df[df['leaid'] == district_to_match]['state_name'].iloc[0]
    
    which_problem = df[df['leaid'] == district_to_match]['problem_tag'].iloc[0]
    
    if district_to_match not in df['leaid'].to_list():
        raise ValueError(f"{district_to_match} not in this problem matrix")
        
    elif isinstance(this_districts_vector, str):
        # provide sample of solutions here instead
        raise ValueError(f"{district_to_match} has no solutions for this problem")

    # have to reshape for cosine similrity calculation
    this_districts_vector = this_districts_vector.reshape(1, -1)

    similarities = {}
    for idx, row in df.iterrows():
        compare_district = row['leaid']
        
        if compare_district == district_to_match or isinstance(row['solution_vector'], str) :
            continue # skip 
        
        compare_district_vector = row['solution_vector'].reshape(1, -1) # reshape for cos sim
        this_cosim = cosine_similarity(this_districts_vector, compare_district_vector)[0][0]
        similarities[compare_district] = this_cosim

    closest_id = max(similarities, key = similarities.get)
    furthest_id = min(similarities, key = similarities.get)

    # get the text solutions associated
    closest_solutions_list = df[df['leaid'] == closest_id]['solutions_list'].iloc[0]
    farthest_solutions_list = df[df['leaid'] == furthest_id]['solutions_list'].iloc[0]

    # get the place name associated
    closest_place = df[df['leaid'] == closest_id]['place_name'].iloc[0]
    farthest_place = df[df['leaid'] == furthest_id]['place_name'].iloc[0]

    # get the state name associated
    closest_state = df[df['leaid'] == closest_id]['state_name'].iloc[0]
    farthest_state = df[df['leaid'] == furthest_id]['state_name'].iloc[0]
    
    close_far_districts = {'which_problem':which_problem,
                           'this_district_leaid':district_to_match,
                           'this_district_place':this_districts_place,
                           'this_district_state':this_districts_state,
                           'this_district_solution':this_districts_solutions,
                           'closest_leaid':closest_id,
                           'closest_place':closest_place,
                           'closest_state':closest_state,
                           'closest_solution':closest_solutions_list,
                           'farthest_leaid':furthest_id,
                           'farthest_place':farthest_place,
                           'farthest_state':farthest_state,
                           'farthest_solution':farthest_solutions_list}
    
    return pd.DataFrame([close_far_districts]) 

#-------------------------------------------------------------
# clean and prepare data for problem grouping

tagged_district_df['solutions_list'] = tagged_district_df['solutions_list'].apply(literal_eval)
tagged_district_df['problem_tag'] = tagged_district_df['problem_tag'].apply(clean_problem_tag)

print('view tagged district dataframe prior to consolidation')
display(tagged_district_df)


#-------------------------------------------------------------
# consolidate by grouping by district and problem tag

# --> get all solutions specific to a given district's specific problem
updated_problem_df = consolidate_district_problem_df(tagged_district_df)

print('view tagged district dataframe after district-tag consolidation')
display(updated_problem_df)    


#-------------------------------------------------------------
# add place metadata

updated_problem_df_w_meta = pd.merge(updated_problem_df,metadata_df, on = 'leaid', how = 'left')
updated_problem_df_w_meta = updated_problem_df_w_meta[['leaid','problem_tag','solutions_list','place_name','state_name','centroid_lat','centroid_lon']] 

print('view w/ district place information')
display(updated_problem_df_w_meta)


#-------------------------------------------------------------
# problem df matrix construction

problem_groups = updated_problem_df_w_meta.groupby('problem_tag')
# ^should be 7 since n was set to 6 in previous step so 6 + 'other' = 7

print('num matrices to construct',len(problem_groups)) 


# within each problem df, get solution vectors for each of the solutions provided in solutions list 

problem_matrix_list = []
for problem, group in problem_groups:
    group['solution_vector'] = group['solutions_list'].apply(get_solution_vectors)
    group = group.loc[group['problem_tag']!='other']
    problem_matrix_list.append(group)

# check what the dfs look like
print('view preliminary prob solution dataset')
for df in problem_matrix_list:
    display(df)

#check vector dim
problem_matrix_list[0]['solution_vector'][0].shape


#-------------------------------------------------------------
# save subset of each df with missing solutions for later

MISSING_SOLUTIONS = [] # list of df where problem mentioned but lacks solution
problem_matrix_list_has_sol = [] # excludes MISSING_SOLUTIONS

for df in problem_matrix_list:
    this_no_solution_df = df[df['solution_vector'].astype(str) == 'NO_SOLUTIONS']
    MISSING_SOLUTIONS.append(this_no_solution_df)
    
    # now can remove them without losing them
    has_solution_df = df[df['solution_vector'].astype(str) != 'NO_SOLUTIONS']
    display(has_solution_df)

    problem_matrix_list_has_sol.append(has_solution_df)

view tagged district dataframe prior to consolidation


,leaid,problem_tag,solutions_list
0,100240,Declining Enrollment and Financial Pressures,[Hold a community meeting to explain situation...
1,100240,Operational and Logistical Challenges,[Commit to a clear decision timeline (by Febru...
2,100240,"Community Trust, Engagement, and Communication...",[Immediately attempt to contact families whose...
3,100240,"Community Trust, Engagement, and Communication...",[Host community meetings to hear and address p...
4,100240,Operational and Logistical Challenges,[Collect preference data from families on desi...
...,...,...,...
471,1601800,"Community Trust, Engagement, and Communication...",[Develop and communicate clear financial and e...
472,1601800,Loss of School and Community Identity,[Conduct up-to-date community surveys to asses...
473,1601800,Declining Enrollment and Financial Pressures,[Evaluate fiscal responsibility by reviewing s...
474,1601800,Operational and Logistical Challenges,"[Develop contingency plans for facility use, i..."


view tagged district dataframe after district-tag consolidation


,leaid,problem_tag,solutions_list
0,100240,"Community Trust, Engagement, and Communication...",[Immediately attempt to contact families whose...
1,100240,Declining Enrollment and Financial Pressures,[Hold a community meeting to explain situation...
2,100240,"Equity, Access, and Student Well-being",[Utilize virtual models for alternative educat...
3,100240,Facilities and Infrastructure Issues,[Initiate a study to analyze restructuring and...
4,100240,Operational and Logistical Challenges,[Commit to a clear decision timeline (by Febru...
...,...,...,...
149,1601800,"Community Trust, Engagement, and Communication...",[Develop and communicate clear financial and e...
150,1601800,Declining Enrollment and Financial Pressures,[Repurpose a school and move students and prog...
151,1601800,Facilities and Infrastructure Issues,[Evaluate the cost-benefit of investing in rep...
152,1601800,Loss of School and Community Identity,[Prioritize exploring alternative options (suc...


view w/ district place information


...

,leaid,problem_tag,solutions_list,place_name,state_name,centroid_lat,centroid_lon,solution_vector
2,100240,"Equity, Access, and Student Well-being",[Utilize virtual models for alternative educat...,Autauga County sb,Alabama,32.534920,-86.642749,"[0.05835192, 0.0018674199, -0.004798067, -0.01..."
9,200180,"Equity, Access, and Student Well-being",[Move schools to facilities with more amenitie...,Anchorage School District sb,Alaska,61.151867,-149.115728,"[0.08798582, 0.065915786, 0.023248062, 0.07475..."
25,600021,"Equity, Access, and Student Well-being",[Reconfigure main office and share workspace a...,Sunol Glen Unified sb,California,37.555113,-121.822062,"[-0.029052308, 0.01053737, -0.020463329, -0.02..."
32,602310,"Equity, Access, and Student Well-being",[Maintain all grade levels of the Vietnamese d...,Alum Rock Union Elementary sb,California,37.355655,-121.729340,"[-0.009109647, -0.047058098, -0.0375861, -0.09..."
38,607440,"Equity, Access, and Student Well-being",[Expand or introduce new district-sponsored pr...,Capistrano Unified sb,California,33.533175,-117.613796,"[0.061526693, -0.045910012, 0.05379349, 0.0203..."
48,613360,"Equity, Access, and Student Well-being","[Develop and fund local, creative solutions to...",Fairfield-Suisun Unified sb,California,38.204942,-122.039066,"[0.019291576, -0.0028993879, 0.018380053, 0.00..."
54,618390,"Equity, Access, and Student Well-being",[Maintain and expand existing special programs...,Inglewood Unified sb,California,33.961456,-118.347915,"[0.013799678, -0.0072879884, 0.07379981, 0.040..."
59,618870,"Equity, Access, and Student Well-being",[Review and communicate the interdistrict tran...,Jefferson Elementary sb,California,37.686221,-122.469221,"[-0.021971863, 0.019459073, 0.026870698, -0.00..."
64,627240,"Equity, Access, and Student Well-being",[Transition students from closed community cen...,Newport-Mesa Unified sb,California,33.636213,-117.888329,"[-0.03733649, -0.010230062, 0.018818565, 0.050..."
72,629490,"Equity, Access, and Student Well-being",[Guarantee placement for all students and staf...,Pajaro Valley Unified sb,California,36.957510,-121.787313,"[0.034766443, -0.020014219, 0.024853762, -0.03..."


,leaid,problem_tag,solutions_list,place_name,state_name,centroid_lat,centroid_lon,solution_vector
3,100240,Facilities and Infrastructure Issues,[Initiate a study to analyze restructuring and...,Autauga County sb,Alabama,32.534920,-86.642749,"[0.0866819, -0.0026515482, 0.0015345068, 0.005..."
6,102820,Facilities and Infrastructure Issues,[Upgrade all existing buildings at once to mai...,Randolph County sb,Alabama,33.298801,-85.462100,"[0.050551027, -0.019706633, 0.03848069, 0.0105..."
10,200180,Facilities and Infrastructure Issues,[Relocate schools to larger facilities such as...,Anchorage School District sb,Alaska,61.151867,-149.115728,"[0.03779352, -0.01450275, -0.0151320435, 0.046..."
19,509000,Facilities and Infrastructure Issues,[Maintain a minimum level of utility usage in ...,Little Rock School District sb,Arkansas,34.721940,-92.349951,"[0.038629904, 0.024548307, 0.0565949, 0.045302..."
22,511970,Facilities and Infrastructure Issues,[Remove all district-needed items such as text...,Rogers School District sb,Arkansas,36.346437,-94.011996,"[-0.004452201, 0.13147523, 0.053537022, 0.0266..."
26,600021,Facilities and Infrastructure Issues,[Plan for relocation of the Health and Wellnes...,Sunol Glen Unified sb,California,37.555113,-121.822062,"[0.05289166, 0.04640938, -0.008881069, 0.07233..."
33,602310,Facilities and Infrastructure Issues,"[Maintain essential utilities at closed sites,...",Alum Rock Union Elementary sb,California,37.355655,-121.729340,"[0.0099949995, 0.037761845, -0.012232699, 0.02..."
39,607440,Facilities and Infrastructure Issues,[Prioritize closing facilities that pose a lia...,Capistrano Unified sb,California,33.533175,-117.613796,"[0.061319556, 0.013028976, 0.065321036, -0.031..."
41,608460,Facilities and Infrastructure Issues,[Request a report detailing current district m...,Chino Valley Unified sb,California,33.969719,-117.695282,"[0.016506458, 0.021094233, 0.044974387, 0.0266..."
50,613890,Facilities and Infrastructure Issues,[Plan and design a combined high school and mi...,Folsom-Cordova Unified sb,California,38.616914,-121.183441,"[-0.010958482, 0.024564184, 0.06728709, -0.065..."


,leaid,problem_tag,solutions_list,place_name,state_name,centroid_lat,centroid_lon,solution_vector
11,200180,Loss of School and Community Identity,[Acknowledge community attachment and communic...,Anchorage School District sb,Alaska,61.151867,-149.115728,"[0.030090367, -0.007437655, -0.01289911, 0.055..."
34,602310,Loss of School and Community Identity,[Preserve or relocate community resource cente...,Alum Rock Union Elementary sb,California,37.355655,-121.729340,"[-0.013029093, -0.037346028, 0.0023097664, 0.0..."
56,618390,Loss of School and Community Identity,[Implement community healing initiatives and r...,Inglewood Unified sb,California,33.961456,-118.347915,"[-0.015977567, 0.0072132796, 0.008010529, 0.00..."
73,629490,Loss of School and Community Identity,[Involve current staff in designing the new pr...,Pajaro Valley Unified sb,California,36.957510,-121.787313,"[0.017676217, -0.044207178, 0.0021816082, -0.0..."
77,629700,Loss of School and Community Identity,[Delay consideration of any cuts to staff and ...,Palos Verdes Peninsula Unified sb,California,33.765965,-118.368669,"[-0.018999612, 0.0049660434, 0.049837038, -0.0..."
81,629940,Loss of School and Community Identity,[Refrain from merging IB coordinators into a s...,Pasadena Unified sb,California,34.195757,-118.095851,"[-0.018119777, -0.020188108, 0.020690301, 0.05..."
99,636800,Loss of School and Community Identity,[Lead with student voice by facilitating focus...,Sierra Sands Unified sb,California,35.562106,-117.775157,"[0.031055452, 0.020558445, -0.01586597, -0.019..."
107,640740,Loss of School and Community Identity,[Engage community stakeholders and representat...,Vallejo City Unified sb,California,38.121127,-122.253194,"[0.015574605, -0.03352723, -0.039971568, 0.023..."
122,806150,Loss of School and Community Identity,[Retain local schools to reinforce community s...,Pueblo County School District 70 sb,Colorado,38.195848,-104.596616,"[0.04941554, 0.043447793, 0.054337986, 0.07227..."
143,1201260,Loss of School and Community Identity,[Engage stakeholders in exploring ways to reco...,Marion sb,Florida,29.209241,-82.057322,"[-0.0071477387, 0.083267, 0.022126213, -0.0080..."


,leaid,problem_tag,solutions_list,place_name,state_name,centroid_lat,centroid_lon,solution_vector
4,100240,Operational and Logistical Challenges,[Commit to a clear decision timeline (by Febru...,Autauga County sb,Alabama,32.534920,-86.642749,"[0.032607045, 0.020533582, 0.010362838, 0.0209..."
14,401460,Operational and Logistical Challenges,[Participate in external studies (such as thos...,Sierra Vista Unified District (4175) sb,Arizona,31.523056,-110.230439,"[0.04538574, -0.034476675, -0.0022843862, 0.07..."
21,509000,Operational and Logistical Challenges,[Standardize the process for staff transitions...,Little Rock School District sb,Arkansas,34.721940,-92.349951,"[-0.019979747, -0.037155006, 0.021483622, -0.0..."
23,511970,Operational and Logistical Challenges,[Enroll majority of former Garfield students a...,Rogers School District sb,Arkansas,36.346437,-94.011996,"[0.004889109, -0.04111737, -0.042162944, -0.00..."
28,600028,Operational and Logistical Challenges,[Consider and approve the bargaining proposal ...,Temecula Valley Unified sb,California,33.526879,-117.050330,"[-0.009791814, 0.03400314, 0.012416934, -0.003..."
35,602310,Operational and Logistical Challenges,[Consolidate and redesign schools to maximize ...,Alum Rock Union Elementary sb,California,37.355655,-121.729340,"[0.02889169, -0.010998051, 0.055303216, 0.0543..."
42,608460,Operational and Logistical Challenges,[Explore strategies to mitigate instructional ...,Chino Valley Unified sb,California,33.969719,-117.695282,"[0.039837305, 0.048051566, 0.0968387, 0.051228..."
51,613890,Operational and Logistical Challenges,[Adjust school attendance boundaries to distri...,Folsom-Cordova Unified sb,California,38.616914,-121.183441,"[0.05056297, 0.032182448, 0.053964674, -0.0118..."
57,618390,Operational and Logistical Challenges,[Use savings from school closures to retain or...,Inglewood Unified sb,California,33.961456,-118.347915,"[-0.03634449, 0.0072501786, 0.0582841, 0.01392..."
63,627060,Operational and Logistical Challenges,[Prioritize filling key leadership positions s...,Newark Unified sb,California,37.520114,-122.030773,"[0.02167458, -0.021505773, -0.020338291, 0.023..."


,leaid,problem_tag,solutions_list,place_name,state_name,centroid_lat,centroid_lon,solution_vector


In [59]:


# randomly sample 2 district problems from each prob matrix and get data

district_result_list = []
for prob_df in problem_matrix_list_has_sol:
    print('num of districts in this problem df: ', len(prob_df))

    if len(prob_df)>=3:
        
        sample_prob_df = prob_df.sample(n=2, random_state = 100)
        sample_list = sample_prob_df['leaid'].to_list()
    
        # for each district in the sample df for this prob, get the closest and farthest solutions
        for district in sample_list:
            result_row_df = find_closest_and_furthest(district_to_match = district, df = prob_df)
            district_result_list.append(result_row_df)
            

final_sample_concatenated_df = pd.concat(district_result_list, axis=0)
display(final_sample_concatenated_df)

final_sample_concatenated_df.to_csv('sample_district_matching.csv')


num of districts in this problem df:  31
num of districts in this problem df:  26
num of districts in this problem df:  23
num of districts in this problem df:  22
num of districts in this problem df:  11
num of districts in this problem df:  23
num of districts in this problem df:  0


,which_problem,this_district_leaid,this_district_place,this_district_state,this_district_solution,closest_leaid,closest_place,closest_state,closest_solution,farthest_leaid,farthest_place,farthest_state,farthest_solution
0,"Community Trust, Engagement, and Communication...",628650,Orange Unified sb,California,[Establish ongoing touch points and updates du...,800260,NaN,Colorado,[Pre-announce recommendations to all affected ...,627060,Newark Unified sb,California,[Implement long-term hiring practices focused ...
0,"Community Trust, Engagement, and Communication...",901710,Greenwich School District sb,Connecticut,[Notify aquatics parents and community stakeho...,612120,NaN,California,[Establish a transparent and consistent commun...,629490,Pajaro Valley Unified sb,California,[Ensure current PCCS teaching staff remain in ...
0,Declining Enrollment and Financial Pressures,612120,NaN,California,[Consider implementing programs to grow revenu...,602310,Alum Rock Union Elementary sb,California,[Reduce special education expenditures by $1.9...,200180,Anchorage School District sb,Alaska,[Undertake a more decisive and thorough school...
0,Declining Enrollment and Financial Pressures,901020,Danbury School District sb,Connecticut,[Plan for how to recoup funding and adjust for...,612120,NaN,California,[Consider implementing programs to grow revenu...,629490,Pajaro Valley Unified sb,California,[Recommend closure of Pacific Coast Charter Sc...
0,"Equity, Access, and Student Well-being",618390,Inglewood Unified sb,California,[Maintain and expand existing special programs...,636800,Sierra Sands Unified sb,California,[Establish an independent study school featuri...,600021,Sunol Glen Unified sb,California,[Reconfigure main office and share workspace a...
0,"Equity, Access, and Student Well-being",806150,Pueblo County School District 70 sb,Colorado,[Keep the middle school open to maintain a dev...,1201590,Polk sb,Florida,[Consider using specific buildings as transiti...,800260,NaN,Colorado,[Include representatives from all departments ...
0,Facilities and Infrastructure Issues,1601800,Lakeland District sb,Idaho,[Evaluate the cost-benefit of investing in rep...,901710,Greenwich School District sb,Connecticut,[Provide reasoning and documentation for why r...,900450,Bridgeport School District sb,Connecticut,[Address underutilization through targeted red...
0,Facilities and Infrastructure Issues,634800,San Luis Coastal Unified sb,California,[Consider extending the school property lease ...,511970,Rogers School District sb,Arkansas,[Remove all district-needed items such as text...,635700,Santa Monica-Malibu Unified sb,California,[Engage professional restoration and cleaning ...
0,Loss of School and Community Identity,1201260,Marion sb,Florida,[Engage stakeholders in exploring ways to reco...,806150,Pueblo County School District 70 sb,Colorado,[Retain local schools to reinforce community s...,629490,Pajaro Valley Unified sb,California,[Involve current staff in designing the new pr...
0,Loss of School and Community Identity,636800,Sierra Sands Unified sb,California,[Lead with student voice by facilitating focus...,629490,Pajaro Valley Unified sb,California,[Involve current staff in designing the new pr...,629940,Pasadena Unified sb,California,[Refrain from merging IB coordinators into a s...
